In [2]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

load_dotenv()

#모델 설정
model = ChatOpenAI(model="gpt-4o-mini")

In [3]:
import pdfplumber
import os
from langchain.schema import Document

# PDF에서 텍스트 추출
docs = []

# 폴더 내 파일 가져오기
path = '교통과산재데이터/'
file_list = os.listdir(path)

for doc_num, file_name in enumerate(file_list):
    file_path = os.path.join(path, file_name)
    try:
        extracted_text = ""  # PDF 한 파일의 전체 텍스트를 저장할 변수
        with pdfplumber.open(file_path) as pdf_file:
            for i, page in enumerate(pdf_file.pages):
                try:
                    # 텍스트 추출
                    text = page.extract_text()
                    if text:
                        extracted_text += text + "\n"  # 각 페이지의 텍스트를 합침
                except Exception as e:
                    print(f"페이지 {i + 1}에서 오류 발생: {e}")

        # 한 PDF 파일의 전체 텍스트를 하나의 Document로 저장
        if extracted_text.strip():  # 추출된 텍스트가 있으면 저장
            document = Document(
                page_content=extracted_text,
                metadata={"doc_number": doc_num + 1, "file_name": file_name}
            )
            docs.append(document)
        else:
            print(f"파일 '{file_name}'에서 텍스트 추출이 없습니다.")

    except Exception as e:
        print(f"파일 '{file_name}'에서 오류 발생: {e}")

# 텍스트 추출 결과 확인
print(f"총 {len(docs)}개의 PDF 파일에서 텍스트를 추출했습니다.")

파일 '2013가합15831_판결문_검수완료.pdf'에서 텍스트 추출이 없습니다.
총 125개의 PDF 파일에서 텍스트를 추출했습니다.


In [4]:
# 이미지로 처리되어 내용이 없는 파일 삭제
print(docs[0].page_content)

docs = docs[1:]

본 판결문은 판결서 인터넷열람 사이트에서 열람·출력되었습니다.
영리목적으로 이용하거나 무단 배포를 금합니다.게시일자 : 2016-10-07
본 판결문은 판결서 인터넷열람 사이트에서 열람·출력되었습니다.
영리목적으로 이용하거나 무단 배포를 금합니다.게시일자 : 2016-10-07
본 판결문은 판결서 인터넷열람 사이트에서 열람·출력되었습니다.
영리목적으로 이용하거나 무단 배포를 금합니다.게시일자 : 2016-10-07
본 판결문은 판결서 인터넷열람 사이트에서 열람·출력되었습니다.
영리목적으로 이용하거나 무단 배포를 금합니다.게시일자 : 2016-10-07
본 판결문은 판결서 인터넷열람 사이트에서 열람·출력되었습니다.
영리목적으로 이용하거나 무단 배포를 금합니다.게시일자 : 2016-10-07
본 판결문은 판결서 인터넷열람 사이트에서 열람·출력되었습니다.
영리목적으로 이용하거나 무단 배포를 금합니다.게시일자 : 2016-10-07
본 판결문은 판결서 인터넷열람 사이트에서 열람·출력되었습니다.
영리목적으로 이용하거나 무단 배포를 금합니다.게시일자 : 2016-10-07
본 판결문은 판결서 인터넷열람 사이트에서 열람·출력되었습니다.
영리목적으로 이용하거나 무단 배포를 금합니다.게시일자 : 2016-10-07
본 판결문은 판결서 인터넷열람 사이트에서 열람·출력되었습니다.
영리목적으로 이용하거나 무단 배포를 금합니다.게시일자 : 2016-10-07
본 판결문은 판결서 인터넷열람 사이트에서 열람·출력되었습니다.
영리목적으로 이용하거나 무단 배포를 금합니다.게시일자 : 2016-10-07
본 판결문은 판결서 인터넷열람 사이트에서 열람·출력되었습니다.
영리목적으로 이용하거나 무단 배포를 금합니다.게시일자 : 2016-10-07
본 판결문은 판결서 인터넷열람 사이트에서 열람·출력되었습니다.
영리목적으로 이용하거나 무단 배포를 금합니다.게시일자 : 2016-10-07



In [5]:
import json

# 과실 비율 데이터 읽어오기
# 폴더 내에 있는 파일 모두 가져오기
with open('accident_data_all_pages.json', 'r', encoding='utf-8') as f:
    file = json.load(f)


# JSON 데이터를 Document로 변환
def nested_json_to_documents(json_data):
    docs = []  # 문서 리스트 초기화
    # 중첩된 리스트를 순회하며 평탄화
    for entry in json_data:  # 최상위 리스트 순회
        content = (
            f"상황: {entry['상황']}\n"
            f"청구인 과실 비율: {entry['청구인 과실 비율']}\n"
            f"피청구인 과실 비율: {entry['피청구인 과실 비율']}\n"
        )
        docs.append(Document(page_content=content))  # Document 객체 추가
    return docs

docs_rate = nested_json_to_documents(file)

In [6]:
from langchain_core.prompts import ChatPromptTemplate

# 사고 상황 요약을 위한 프롬프트 템플릿 정의
summary_prompt = ChatPromptTemplate.from_messages([
    ('system', '주어진 문서 내의 "교통사고 발생 상황"을 "사고 원인"을 포함해서 한 문장으로 요약해줘. 단순 사고 상황에 대한 내용만 다루고 사건에 대한 판결의 내용은 넣지 말아줘.'),
    ('user', '{content}')
])

# 사고 상황을 요약하는 함수 (LLM 모델 사용)
def summarize_accident(accident_text):
    summary = summary_prompt.format_messages(content=accident_text)
    result = model.invoke(summary)
    return result  # 요약된 사고 상황 반환


# 문서 요약본 새 document로 저장
def summary_docs(original_doc, summary_text):
    updated_doc = Document(
        metadata={**original_doc.metadata, 'summary': summary_text},  # 요약 추가
        page_content=original_doc.page_content,
    )
    return updated_doc


# 요약된 문서들을 저장할 리스트
summarized_docs = []

# 모든 문서에 대해 요약 생성, 저장 및 임베딩
for i in range(len(docs)):
    summary = summarize_accident(docs[i].page_content)  # 요약 생성
    sum_doc = summary_docs(docs[i], summary)  # 요약 추가
    summarized_docs.append(sum_doc)  # 리스트에 추가

In [7]:
from langchain_openai import OpenAIEmbeddings
import faiss
from langchain_community.vectorstores import FAISS
from uuid import uuid4

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# 문서 별 고유 ID 생성
uuids1 = [f"docs_{i+1}" for i in range(len(summarized_docs))]
uuids2 = [f"docs_{i+1}" for i in range(len(docs_rate))]

# FAISS 벡터 스토어 생성
vector_store_situation = FAISS.from_documents(
    documents=summarized_docs, ids=uuids1, embedding=embeddings)
vector_store_rate = FAISS.from_documents(
    documents=docs_rate, ids=uuids2, embedding=embeddings)

In [8]:
# 유사성 검색 리트리버 정의
retriever1 = vector_store_situation.as_retriever(
    search_type="similarity", search_kwargs={"k": 1})
retriever2 = vector_store_rate.as_retriever(
    search_type="similarity", search_kwargs={"k": 1})

In [31]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# 프롬프트 템플릿 정의
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", '''
귀하는 교통사고 과실 비율을 판단하는 챗봇입니다.
다음으로 제공되는 content1은 비슷한 사고에 관한 법원의 "판결문"이므로 사고 과실 비율 판단에 참고하세요.
    또한, content2는 비슷한 사고에 관한 법원에서 인정된 "과실 비율"이므로, Question에 대한 과실 비율 판단에 참고하세요.


준수해야 할 규칙:
1. 사용자가 입력한 사고 상황을 이해한 후, 이해한 상황을 사용자에게 안내하세요.
2. 법에 따라 판단해야 합니다.
3. 판결 결과를 안내하면서 동시에 실제 판레도 같이 언급하세요. (되도록 content1의 법원의 판단, content2의 과실 비율 모두 언급하세요.)
4. 판단이 올바르지 않을 가능성이 있으므로 전문가와 상의하여 보다 상세하고 신뢰할 수 있는 판단을 내릴 수 있도록 안내해 주시기 바랍니다.
5. 판단을 내려야 할 상황에서 사용자의 입장이 불확실하다면 반드시 사용자에게 확인하세요.
6. 주어진 상황에 관련된 법률에 대한 정보가 없다면 모른다고 대답하세요.
7. 만일 content2의 사고 상황이 입력된 사고 상황과 유사하지 않다고 판단된다면, 관련 사례가 없음을 안내한 뒤 content1의 판결문을 참고해 과실 비율을 판단하세요.
8. 응답을 시작할 때, 사고 상황을 겪은 사용자를 위로해주는 말로 대화를 시작하세요.

주의할 규칙:
1. 사고 상황에 대해 정리할 때에는 반드시 question의 내용으로만 정리하세요.
2. 과실 비율에 대해 판단 할 때에는 기본적으로 content1과 content2의 정보 모두 "참고"하세요. (상황이 유사하지 않은 경우, 정보 없는 경우와 같은 부득이한 경우 제외)

예시 응답:
- "제공된 사고 상황에서 귀하께선 어떠한 입장이십니까?"
- "사고 상황 : 고속도로에서 갑작스럽게 후진으로 인한 사고

    가능한 과실 비율 : 귀하의 경우에는 고속도로에서 갑작스럽게 후진하셨으므로 과실 비율은 1 : 9로 귀하가 최대 1,000,000원의 합의금을 내야할 수 있습니다.

    실제 판례 : [상황] 고속도로에서의 후진으로 인한 사고 [청구인의 과실 비율] 10% [피청구인의 과실 비율] 90%"


- "죄송합니다. 사고와 관련된 법률에 대한 정보가 없기 때문에 판단이 불가능합니다."
    
응답 형태:
    
    '''),
    ("user", "Context1: {context1}\\n\\Context2: {context2}\\n\\nQuestion: {question}")
])

In [10]:
from langchain.chains import LLMChain


class SimplePassThrough:
    def invoke(self, inputs, **kwargs):
        return inputs


class ContextToPrompt:
    def __init__(self, prompt_template):
        self.prompt_template = prompt_template

    def invoke(self, inputs):
        # 문서 내용을 텍스트로 변환
        if isinstance(inputs, list):
            context_text = "\n".join([doc.page_content for doc in inputs])
        else:
            context_text = inputs

        # 프롬프트 템플릿에 적용
        formatted_prompt = self.prompt_template.format_messages(
            context=context_text,
            question=inputs.get("question", "")
        )
        return formatted_prompt

# Retriever를 invoke() 메서드로 래핑하는 클래스 정의
class RetrieverWrapper:
    def __init__(self, retriever):
        self.retriever = retriever

    def invoke(self, inputs):
        if isinstance(inputs, dict):
            query = inputs.get("question", "")
        else:
            query = inputs
        # 검색 수행
        response_docs = self.retriever.get_relevant_documents(query)
        return response_docs

# RAG 체인 구성
rag_chain_debug = {
    "context1": RetrieverWrapper(retriever1),
    'context2': RetrieverWrapper(retriever2),
    "prompt": ContextToPrompt(contextual_prompt),
    "llm": model
}

In [17]:
# 비슷한 상황에서 판결된 과실 비율 문서 검색
from difflib import SequenceMatcher

def get_similarity(a, b):  # 두 문자열 간의 순서대로 일치하는 부분을 비교하여 유사도를 계산
    return SequenceMatcher(None, a, b).ratio()  # 문자열 a와 b 간의 유사도를 계산

def find_most_similar_doc(user_accident):
    max_similarity = 0
    best_match = None # 가장 유사한 문서 저장할 변수
    for doc in docs_rate:
        accident_situation = doc.page_content
        similarity = get_similarity(user_accident, accident_situation)
        if similarity > max_similarity:
            max_similarity = similarity
            best_match = doc
    return best_match

In [ ]:
# 챗봇 구동
query = ''
while True:
    print("========================")
    query = input("질문을 입력하세요 : ")

    if query == 'stop':
        print("RAG 실행을 종료합니다.")
        break

    # 1. Retriever로 관련 문서 검색
    response_docs = rag_chain_debug["context1"].invoke({"question": query})

    # 1-1. 관련 문서와 관련된 과실 비율 문서 검색
    response_docs2 = find_most_similar_doc(response_docs[0].metadata['summary'].content)

    # 2. 문서를 프롬프트로 변환
    prompt_messages = contextual_prompt.format_messages(
        context1=response_docs[0].page_content,
        context2=response_docs2.page_content,
        question=query
    )

    # 3. LLM으로 응답 생성
    response = rag_chain_debug["llm"].invoke(prompt_messages)

    print("\n답변:")
    print(response.content)

In [19]:
from difflib import SequenceMatcher

def get_similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

def find_most_similar_doc(user_accident):
    max_similarity = 0
    best_match = None
    for doc in docs_rate:
        accident_situation = doc.page_content
        similarity = get_similarity(user_accident, accident_situation)
        if similarity > max_similarity:
            max_similarity = similarity
            best_match = doc
    return best_match


# 챗봇 구동
query = ''
while True:
    print("========================")
    query = input("질문을 입력하세요 : ")

    if query == 'stop':
        print("RAG 실행을 종료합니다.")
        break

    # 1. Retriever로 관련 문서 검색
    response_docs = rag_chain_debug["context1"].invoke({"question": query})

    # 1-1. 관련 문서와 관련된 과실 비율 문서 검색
    response_docs2 = find_most_similar_doc(response_docs[0].metadata['summary'].content)

RAG 실행을 종료합니다.


In [21]:
response_docs2.page_content

'상황: 편도4차선도로 3차로 직진차량과 4차로에서 3차로로 진로변경차량간 사고\n청구인 과실 비율: 30%\n피청구인 과실 비율: 70%\n'